In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import random
import numpy as np

In [ ]:
def yolo2standard(bboxes):
     xmin = bboxes[0]-bboxes[2]/2
     xmax = bboxes[0]+bboxes[2]/2
     ymin = bboxes[1]-bboxes[3]/2
     ymax = bboxes[1]+bboxes[3]/2
     return xmin, ymin, xmax, ymax

In [ ]:
class_names = [ 'Ambulance', 'Bus', 'Car','Motorcycle', 'Truck']
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

def plot_box(image, bboxes, labels):
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2standard(box)
        xmin = int(x1*w)
        xmax = int(x2*w)
        ymin = int(y1*h)
        ymax = int(y2*h)
        width = xmin - xmax
        height = ymin - ymax
        
        class_name = class_names[int(labels[box_num])]
        
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=colors[class_names.index(class_name)],
            thickness=2
        ) 
        
        font_scale = min(1,max(3,int(w/500)))
        font_thickness = min(2, max(10,int(w/50)))
        
        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
        
        # Text width and height
        tw, th = cv2.getTextSize(class_name, 0, fontScale=font_scale, thickness=font_thickness)[0]
        p2 = p1[0] + tw, p1[1] + -th - 10
        cv2.rectangle(
            image, 
            p1, p2,
            color=colors[class_names.index(class_name)],
            thickness=-1,
        )
        cv2.putText(
            image, 
            class_name,
            (xmin+1, ymin-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            font_thickness
        )
    return image

In [ ]:
def plot(image_paths, label_paths, num_samples):
    all_training_images = glob.glob(image_paths)
    all_training_labels = glob.glob(label_paths)
    all_training_images.sort()
    all_training_labels.sort()
    
    num_images = len(all_training_images)
    
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image = cv2.imread(all_training_images[j])
        with open(all_training_labels[j], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=0)
    plt.tight_layout()
    plt.show()

In [ ]:
plot(image_paths='/kaggle/input/cars-detection/Cars Detection/train/images/*',
    label_paths='/kaggle/input/cars-detection/Cars Detection/train/labels/*',
    num_samples=4,
)

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")

In [ ]:
results = model.train(data = "/kaggle/input/yaml-5/data.yaml", epochs=100)


In [ ]:
metrics = model.val()

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt source='/kaggle/input/cars-detection/Cars Detection/test/images/3429c6851095a4c3_jpg.rf.2921b5c17bda3bda8b69e0f8b5e44894.jpg'

In [ ]:
img_path = '/kaggle/working/runs/detect/predict/3429c6851095a4c3_jpg.rf.2921b5c17bda3bda8b69e0f8b5e44894.jpg'
img = plt.imread(img_path)

plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis(False)
plt.show()

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt source='/kaggle/input/test-3/test-2.mp4'

In [ ]:
import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [ ]:
# Convert avi to mp4
!ffmpeg -y -loglevel panic -i /kaggle/working/runs/detect/predict5/test-2.avi /kaggle/working/output.mp4

# output object tracking video
path_output = "/kaggle/working/output.mp4"
show_video(path_output, width=960)